Topic modeling is a part of natural language processing (NLP) which enables end-users to identify themes and topics within a collection of documents. It has applications in multiple industries for text mining and gaining relevant insights from textual data. This talk will cover the math behind topic modeling, one example of extracting topics from a sample data and how the user can leverage these outcomes to take actionable insights. This talk will also discuss the necessary infrastructure for an enterprise-wide product and how multiple teams collaborate together on such solutions.

Topic modeling is a technique for determining the topics in a document. It can ve also used to discover patterns of words in a collection of documents. Analysing the frequency of words and phrases in the documents determines the probability of a word or phrase belonging to a certain topic and cluster documents based on their similarity or closeness.

Latent Dirichlet Allocation (LDA) is an unsupervised clustering technique that is commonly used for text analysis. Here, words are represented as topics and documents are collection of these word topics. 

## Dataset

Researchers have access to large online archives of scientific articles. As a consequence, finding relevant articles has become more difficult. Tagging or topic modelling provides a way to give token of identification to research articles which facilitates recommendation and search process.

Given the abstract and title for a set of research articles, predict the topics for each article included in the test set.

Note that a research article can possibly have more than 1 topic. The research article abstracts and titles are sourced from the following 6 topics:

Computer Science

Physics

Mathematics

Statistics

Quantitative Biology

Quantitative Finance

Acknowledgements

Thanks to Analytics Vidhya for the Dataset

Link to Kaggle - https://www.kaggle.com/datasets/blessondensil294/topic-modeling-for-research-articles?resource=download&select=train.csv

In [1]:
import pandas as pd

In [1]:
f1 = pd.read_csv('archive/train.csv')
f2 = pd.read_csv('archive/test.csv')

In [4]:
f2.head()

,ID,TITLE,ABSTRACT
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...,We present novel understandings of the Gamma...
1,20974,Laboratory mid-IR spectra of equilibrated and ...,Meteorites contain minerals from Solar Syste...
2,20975,Case For Static AMSDU Aggregation in WLANs,Frame aggregation is a mechanism by which mu...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...,Milky Way open clusters are very diverse in ...
4,20977,Witness-Functions versus Interpretation-Functi...,Proving that a cryptographic protocol is cor...


In [6]:
f1

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [1]:
# spark packages
# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StopWordsRemover
# import sparknlp
# from sparknlp.base import *
# from sparknlp.annotator import *

# import re
# import pandas as pd
# import numpy as np
# import random

# Warnings
# import warnings
# warnings.filterwarnings('ignore', category = DeprecationWarning)

# from gensim.corpora import Dictionary
# from gensim.models import LdaModel